In [223]:
import json
import pandas as pd
import torch.nn as nn
import torch
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cpu'
device

'cpu'

In [224]:
jsonl_file_path = 'clips_dataset_wth_herbert_token_ids.jsonl'


In [225]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# Assuming CUDA is available, models and data will be moved to GPU for faster processing.
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 33 * 3  # 33 joints * 3 coordinates
hidden_size = 256
output_size = 50000 # Assuming a vocabulary size of 66
embedding_size = 256  # Make embedding size equal to hidden_size for simplicity
SOS_token = 0  # Start-of-Sequence token

In [226]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS": 0, "EOS": 1}
        self.word2count = {"SOS": 0, "EOS": 0}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2   # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

class SignLanguage:
    def __init__(self):
        self.frames = []
        self.max_length = 0

    def addFrames(self, frame_data):
        self.frames.append(frame_data)
        if len(frame_data) > self.max_length:
            self.max_length = len(frame_data)

    def getFrames(self):
        return self.frames


In [227]:
def prepareData(jsonl_file_path, max_samples=1000):
    data = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
            if len(data) >= max_samples:
                break

    def processData(data):
        sign_language = SignLanguage()
        lang_annotations = Lang('pl')
        for item in data:
            sign_language.addFrames(item['FramesLandmarksCoords'])
            lang_annotations.addSentence(item['PolishAnnotation'])
        return sign_language, lang_annotations, [(item['FramesLandmarksCoords'], item['PolishAnnotation']) for item in data]

    return processData(data)


In [228]:
training_dataset = prepareData(jsonl_file_path, max_samples=2000)

train_sign_language, train_polish_annotations, train_pairs = training_dataset

print(f"Loaded {len(train_pairs)} pairs of sign language frames and Polish annotations.")

Loaded 2000 pairs of sign language frames and Polish annotations.


In [229]:
EOS_token = 1

In [230]:
if 'EOS' not in train_polish_annotations.word2index:
    train_polish_annotations.addWord('EOS')


In [231]:
# from torch.utils.data import DataLoader, random_split

# def tensorsFromPair(pair, annotations):
#     # Attempt to handle a variety of data structures defensively
#     try:
#         # Flatten the frame data correctly, assuming it's deeply nested
#         flat_list = [coord for frame in pair[0] for landmark in frame for coord in (landmark if isinstance(landmark, list) else [landmark])]
#     except TypeError:
#         # If there's a type error, log the problematic data and skip or handle appropriately
#         print("Error processing pair:", pair)
#         flat_list = []

#     # Now, calculate the dimensions based on the corrected flat list
#     if flat_list:
#         input_tensor = torch.tensor(flat_list, dtype=torch.float).view(-1, len(flat_list) // len(pair[0]))  # Adjust based on actual data length
#     else:
#         # Handle the error case (e.g., return a dummy tensor or skip this pair)
#         input_tensor = torch.tensor([], dtype=torch.float).view(-1, 1)  # Dummy tensor, adjust as needed

#     # Process target indexes
#     target_indexes = [annotations.word2index[word] for word in pair[1].split(' ')] + [EOS_token]
#     target_tensor = torch.tensor(target_indexes, dtype=torch.long)

#     return (input_tensor, target_tensor)

# from torch.nn.utils.rnn import pad_sequence

# def get_dataloader(pairs, batch_size, annotations):
#     input_tensors = []
#     target_tensors = []

#     for pair in pairs:
#         input_tensor, target_tensor = tensorsFromPair(pair, annotations)
#         input_tensors.append(input_tensor)
#         target_tensors.append(target_tensor)

#     input_tensors_padded = pad_sequence(input_tensors, batch_first=True, padding_value=0)
#     target_tensors_padded = pad_sequence(target_tensors, batch_first=True, padding_value=annotations.word2index['EOS'])

#     dataset = TensorDataset(input_tensors_padded, target_tensors_padded)
    
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     return dataloader

# # Function to split dataset and create dataloaders for each subset
# def create_train_test_dataloaders(pairs, batch_size, annotations, split_ratio=0.5):
#     # Calculate split sizes
#     dataset_size = len(pairs)
#     train_size = int(dataset_size * split_ratio)
#     test_size = dataset_size - train_size

#     # Split pairs dataset
#     train_pairs, test_pairs = random_split(pairs, [train_size, test_size])

#     # Create DataLoader for each subset
#     train_loader = get_dataloader(list(train_pairs), batch_size, annotations)
#     test_loader = get_dataloader(list(test_pairs), batch_size, annotations)

#     return train_loader, test_loader

# # Example usage
# # Assuming 'train_pairs' is your dataset and 'annotations' is initialized
# batch_size = 64
# train_loader, test_loader = create_train_test_dataloaders(train_pairs, batch_size, train_polish_annotations)


In [232]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch
from torch.nn.utils.rnn import pad_sequence
import random

# Assuming EOS_token is defined somewhere
EOS_token = 1  # Example value, adjust according to your EOS token's index

def tensorsFromPair(pair, annotations):
    try:
        flat_list = [coord for frame in pair[0] for landmark in frame for coord in (landmark if isinstance(landmark, list) else [landmark])]
    except TypeError:
        print("Error processing pair:", pair)
        flat_list = []

    if flat_list:
        input_tensor = torch.tensor(flat_list, dtype=torch.float).view(-1, len(flat_list) // len(pair[0]))
    else:
        input_tensor = torch.tensor([], dtype=torch.float).view(-1, 1)

    target_indexes = [annotations.word2index[word] for word in pair[1].split(' ')] + [EOS_token]
    target_tensor = torch.tensor(target_indexes, dtype=torch.long)

    return input_tensor, target_tensor

def get_dataloader(pairs, batch_size, annotations, max_target_length):
    input_tensors = []
    target_tensors = []

    for pair in pairs:
        input_tensor, target_tensor = tensorsFromPair(pair, annotations)
        input_tensors.append(input_tensor)
        # Manually pad target_tensor to max_target_length
        padded_target_tensor = torch.cat([target_tensor, torch.full((max_target_length - target_tensor.size(0),), annotations.word2index['EOS'], dtype=torch.long)])
        target_tensors.append(padded_target_tensor)

    input_tensors_padded = pad_sequence(input_tensors, batch_first=True, padding_value=0)
    target_tensors_padded = torch.stack(target_tensors, dim=0)  # Already manually padded

    dataset = TensorDataset(input_tensors_padded, target_tensors_padded)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return dataloader

def create_train_test_dataloaders(pairs, batch_size, annotations, split_ratio=0.8):
    # Calculate the maximum target tensor length from all pairs
    max_target_length = max(len(pair[1].split(' ')) for pair in pairs) + 1  # +1 for EOS_token

    # Split the dataset
    dataset_size = len(pairs)
    train_size = int(dataset_size * split_ratio)
    test_size = dataset_size - train_size
    train_pairs, test_pairs = random_split(pairs, [train_size, test_size])

    # Create DataLoader for each subset, now passing max_target_length
    train_loader = get_dataloader(list(train_pairs), batch_size, annotations, max_target_length)
    test_loader = get_dataloader(list(test_pairs), batch_size, annotations, max_target_length)

    return train_loader, test_loader

# Example usage:
# Assuming 'train_pairs' is your dataset and 'annotations' is initialized with a 'word2index' mapping
batch_size = 64
train_loader, test_loader = create_train_test_dataloaders(train_pairs, batch_size, train_polish_annotations)


In [233]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        # Applying dropout to the input might not be directly applicable if input is continuous.
        # Consider applying dropout after GRU if needed or using it in a different manner.
        output, hidden = self.gru(input)
        return output, hidden

In [234]:
MAX_LEN = 0

for input_tensor, target_tensor in train_loader:
    if target_tensor.size()[1] > MAX_LEN:
        MAX_LEN = target_tensor.size()[1]

In [235]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LEN):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None and i < target_tensor.size(1) - 1:
            # Teacher forcing: Feed the target as the next input, ensuring we don't exceed target_tensor's length
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [236]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        query = query.float()
        keys = keys.float()
        scores = self.Va(torch.tanh(self.Wa(query.expand(-1, keys.size(1), -1)) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LEN):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [237]:
input_size = 33 * 3  # Assuming each frame is represented by 33 landmarks, each with 3 coordinates
hidden_size = 256  # Example hidden size, adjust based on model complexity and dataset
output_size = len(train_polish_annotations.word2index)
dropout_p = 0.1  # Dropout probability

encoder = EncoderRNN(input_size, hidden_size, dropout_p).to(device)
decoder = AttnDecoderRNN(hidden_size, output_size, dropout_p).to(device)

# Example of model initialization
print(encoder)
print(decoder)

EncoderRNN(
  (gru): GRU(99, 256, batch_first=True)
  (dropout): Dropout(p=0.1, inplace=False)
)
AttnDecoderRNN(
  (embedding): Embedding(4515, 256)
  (attention): BahdanauAttention(
    (Wa): Linear(in_features=256, out_features=256, bias=True)
    (Ua): Linear(in_features=256, out_features=256, bias=True)
    (Va): Linear(in_features=256, out_features=1, bias=True)
  )
  (gru): GRU(512, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=4515, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [238]:
import random

def decode_sentence(indices, lang):
    return ' '.join([lang.index2word.get(index, '') for index in indices if index not in (EOS_token,)])

def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder.train()
    decoder.train()

    total_loss = 0
    total_correct = 0
    total_samples = 0

    for input_tensor, target_tensor in dataloader:
        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        # Compute loss
        loss = criterion(decoder_outputs.view(-1, decoder_outputs.size(-1)), target_tensor.view(-1))
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy excluding padding
        _, predicted = decoder_outputs.max(-1)  # Get the index of the max log-probability
        non_padding_mask = target_tensor != EOS_token
        correct = (predicted == target_tensor) & non_padding_mask  # Apply mask
        num_correct = correct.sum().item()
        total_samples += non_padding_mask.sum().item()  # Count non-padding values only

        total_correct += num_correct

    # Select a random example from the last batch processed for printing
    example_index = random.randint(0, input_tensor.size(0) - 1)  # Randomly select an index
    predicted_example = predicted[example_index]
    target_example = target_tensor[example_index]

    predicted_sentence = decode_sentence(predicted_example[predicted_example != EOS_token].cpu().numpy(), train_polish_annotations)
    target_sentence = decode_sentence(target_example[target_example != EOS_token].cpu().numpy(), train_polish_annotations)
    
    print("Predicted sentence:", predicted_sentence)
    print("Target sentence:", target_sentence)

    average_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples if total_samples > 0 else 0

    return average_loss, accuracy





In [239]:
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.0001)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.0001)
criterion = nn.NLLLoss(ignore_index=train_polish_annotations.word2index['EOS']).to(device)


In [240]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()

In [241]:
def evaluate(dataloader, encoder, decoder, criterion):

    total_loss = 0
    total_correct = 0
    total_samples = 0
    
    with torch.no_grad():  # Disable gradient calculation
        for input_tensor, target_tensor in dataloader:
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

            loss = criterion(decoder_outputs.view(-1, decoder_outputs.size(-1)), target_tensor.view(-1))
            total_loss += loss.item()

            # Calculate accuracy excluding padding
            _, predicted = decoder_outputs.max(-1)  # Get the index of the max log-probability
            non_padding_mask = target_tensor != EOS_token
            correct = (predicted == target_tensor) & non_padding_mask  # Apply mask
            num_correct = correct.sum().item()
            total_samples += non_padding_mask.sum().item()  # Count non-padding values only

            total_correct += num_correct

    average_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_samples if total_samples > 0 else 0
    
    return average_loss, accuracy

In [242]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


def train(train_dataloader, test_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    print_acc_total = 0

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss().to(device)

    for epoch in range(1, n_epochs + 1):
        loss, accuracy = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        print_acc_total += accuracy
        plot_loss_total += loss

        test_loss, test_accuracy = evaluate(test_dataloader, encoder, decoder, criterion)
        
        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_acc_avg = print_acc_total / print_every
            print_loss_total = 0
            print_acc_total = 0
            print('%s (%d %d%%) Train Loss: %.4f Train Acc: %.2f%% Test Loss: %.4f Test Acc: %.2f%%' % (timeSince(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg, print_acc_avg * 100, test_loss, test_accuracy * 100))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    print(plot_losses)
    showPlot(plot_losses)


In [244]:
train(train_loader,test_loader, encoder, decoder, 15, print_every=1, plot_every=1)

torch.Size([64, 467, 99]) torch.Size([64, 42])
torch.Size([64, 467, 99]) torch.Size([64, 42])
torch.Size([64, 467, 99]) torch.Size([64, 42])


KeyboardInterrupt: 